In [1]:
import pandas as pd

data = pd.read_csv("../../data/phemernr1_dataset_with_tvt.csv", lineterminator="\n")
print(data.shape)
data.head()

(5802, 6)


,tweet_text,topic,label,tvt,cv_fold,tvt2
0,BOMBSHELL: #Ferguson chief says the police off...,ferguson,rumours,test,3,training
1,It appears that #Ferguson PD are trying to ass...,ferguson,rumours,training,1,training
2,"All weekend ppl will be talking about the ""rob...",ferguson,rumours,test,3,training
3,Why would the officer tell #MikeBrown to get o...,ferguson,rumours,test,3,training
4,Michael Brown is the 17 yr old boy who was sho...,ferguson,rumours,training,0,testting


In [2]:
length = data.shape[0]
length

5802

In [3]:
import random

# random.seed(33)

train_w = [0 for i in range(675)]
val_w = [1 for i in range(225)]
test_w = [2 for i in range(100)]

weights = train_w + val_w + test_w
print(f"weights : {len(weights)}")

tvt = []
for i in range(length):
    gacha = random.sample(weights, 1)[0]
    if gacha == 0:
        tvt.append("training")
    elif gacha == 1:
        tvt.append("validation")
    else:
        tvt.append("testting")

print(f"Training : {tvt.count('training')} - {round(tvt.count('training')/length, 3)}")
print(f"Validation : {tvt.count('validation')} - {round(tvt.count('validation')/length, 3)}")
print(f"Testing : {tvt.count('testting')} - {round(tvt.count('testting')/length, 3)}")

weights : 1000
Training : 3841 - 0.662
Validation : 1350 - 0.233
Testing : 611 - 0.105


In [4]:
data['tvt2'] = pd.Series(tvt)
data.head()

,tweet_text,topic,label,tvt,cv_fold,tvt2
0,BOMBSHELL: #Ferguson chief says the police off...,ferguson,rumours,test,3,validation
1,It appears that #Ferguson PD are trying to ass...,ferguson,rumours,training,1,training
2,"All weekend ppl will be talking about the ""rob...",ferguson,rumours,test,3,training
3,Why would the officer tell #MikeBrown to get o...,ferguson,rumours,test,3,validation
4,Michael Brown is the 17 yr old boy who was sho...,ferguson,rumours,training,0,training


In [5]:
label_count = data['label'].value_counts()
label_count

non-rumours    3830
rumours        1972
Name: label, dtype: int64

In [6]:
topic_count = data['topic'].value_counts()
combination = data.apply(lambda row: f"{row['topic']}_{row['tvt2']}", axis=1).value_counts()
# print(combination)
comparison = {}
for k, comb in combination.items():
    cv_fold = k.split("_")[1]
    label = k.split("_")[0]

    if cv_fold not in comparison:
        comparison[cv_fold] = {}
    
    comparison[cv_fold][label] = comb

topics = data['topic'].unique().tolist()
topics_simple = [t.split("-")[0] for t in topics]

def label_ratio(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{round(label_dict[l]/total, 4)}\t"
    
    return report[:-1]

def label_raw_value(label_dict, labels):
    report = ""
    for l in labels:
        report += f"{label_dict[l]}\t"
    
    return report[:-1]

topics_str = ','.join(topics_simple)
print(f"\nTopic,{topics_str}")
print(f"Original\t{label_ratio(topic_count, topics)}")
for cv, comp in comparison.items():
    print(f"Part-{cv}\t{label_ratio(comp, topics)}")

topics_str = ','.join(topics_simple)
print(f"\nTopic,{topics_str}")
print(f"Original\t{label_raw_value(topic_count, topics)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_raw_value(comp, topics)}")


Topic,ferguson,charliehebdo,germanwings,ottawashooting,sydneysiege
Original	0.197	0.3583	0.0808	0.1534	0.2104
Part-training	0.1927	0.3637	0.0802	0.1515	0.2119
Part-validation	0.2104	0.3526	0.0837	0.157	0.1963
Part-testting	0.1948	0.3372	0.0786	0.1571	0.2324

Topic,ferguson,charliehebdo,germanwings,ottawashooting,sydneysiege
Original	1143	2079	469	890	1221
Training	740	1397	308	582	814
Validation	284	476	113	212	265
Testting	119	206	48	96	142


In [7]:
combination = data.apply(lambda row: f"{row['label']}_{row['tvt2']}", axis=1).value_counts()
comparison = {}
for k, comb in combination.items():
    cv_fold = k.split("_")[1]
    label = k.split("_")[0]

    if cv_fold not in comparison:
        comparison[cv_fold] = {}
    
    comparison[cv_fold][label] = comb

labels = data['label'].unique().tolist()

def label_ratio(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{round(label_dict[l]/total, 2)}\t"
    
    return report[:-1]

def label_raw_value(label_dict, labels):
    total = sum([v for k, v in label_dict.items()])
    
    report = ""
    for l in labels:
        report += f"{label_dict[l]}\t"
    
    return report[:-1]

labels_str = ','.join(labels)
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_ratio(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_ratio(comp, labels)}")
    
labels_str = ','.join(labels)
print(f"\nLabel,{labels_str}")
print(f"Original\t{label_raw_value(label_count, labels)}")
for cv, comp in comparison.items():
    print(f"{cv.title()}\t{label_raw_value(comp, labels)}")


Label,rumours,non-rumours
Original	0.34	0.66
Training	0.34	0.66
Validation	0.33	0.67
Testting	0.34	0.66

Label,rumours,non-rumours
Original	1972	3830
Training	1313	2528
Validation	451	899
Testting	208	403


In [9]:
data.to_csv("../../data/phemernr1_dataset_with_tvt.csv", index=False)